In [ ]:
# pip install praw python-dotenv

In [1]:
import praw
import os
from dotenv import load_dotenv
import time
import json

In [2]:

# Load environment variables
load_dotenv()

# Initialize Reddit API client
reddit = praw.Reddit(
    client_id=os.getenv('REDDIT_CLIENT_ID'),
    client_secret=os.getenv('REDDIT_CLIENT_SECRET'),
    user_agent=os.getenv('REDDIT_USER_AGENT')
)


In [4]:
subreddit = reddit.subreddit("JapanTravel")

for post in subreddit.new(limit=1):
    print(post)

TypeError: BaseListingMixin.new() takes 1 positional argument but 2 were given

In [ ]:
def scrape_subreddit(subreddit_name, limit=None):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []

    for post in subreddit.new(limit=limit):
        post_data = {
            'id': post.id,
            'title': post.title,
            'author': post.author.name if post.author else '[deleted]',
            'created_utc': post.created_utc,
            'score': post.score,
            'body': post.selftext,
            'url': post.url,
            'num_comments': post.num_comments
        }
        
        # Fetch comments
        post.comments.replace_more(limit=0)  # Flatten comment tree
        comments = []
        for comment in post.comments.list():
            comment_data = {
                'id': comment.id,
                'author': comment.author.name if comment.author else '[deleted]',
                'body': comment.body,
                'score': comment.score,
                'created_utc': comment.created_utc
            }
            comments.append(comment_data)
        
        post_data['comments'] = comments
        
        posts.append(post_data)
        print(f"Scraped post: {post.id}")
        
        # Sleep to respect Reddit's rate limits
        time.sleep(2)
    
    return posts